<a href="https://www.kaggle.com/code/swish9/multi-class-classification-of-fruits-using-tf?scriptVersionId=179184873" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import GlorotNormal

In [3]:
train_dir = '/kaggle/input/fruits-dataset-images/images'

In [4]:
img_width, img_height = 150, 150

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 359 images belonging to 9 classes.


In [7]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(9, activation='softmax')
])

**I tried using He and Glorot Initialization techniques for weights, but I was getting accuracies around 0.6 to 0.7. However, with random initialization, my highest accuracy was 0.84**

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
lr_sch = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [10]:
num_epochs = 100
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    verbose=1,
    callbacks=[lr_sch]
)

Epoch 1/100
12/12 [==============================] - 24s 2s/step - loss: 3.2658 - accuracy: 0.1950 - lr: 0.0010
Epoch 2/100
12/12 [==============================] - 12s 1s/step - loss: 2.0423 - accuracy: 0.3844 - lr: 0.0010
Epoch 3/100
12/12 [==============================] - 12s 1s/step - loss: 2.9312 - accuracy: 0.4178 - lr: 0.0010
Epoch 4/100
12/12 [==============================] - 12s 1s/step - loss: 3.7422 - accuracy: 0.3900 - lr: 0.0010
Epoch 5/100
12/12 [==============================] - 13s 1s/step - loss: 3.1499 - accuracy: 0.4791 - lr: 0.0010
Epoch 6/100
12/12 [==============================] - 13s 1s/step - loss: 3.5997 - accuracy: 0.4735 - lr: 0.0010
Epoch 7/100
12/12 [==============================] - 12s 1s/step - loss: 1.8102 - accuracy: 0.5070 - lr: 0.0010
Epoch 8/100
12/12 [==============================] - 13s 1s/step - loss: 2.9968 - accuracy: 0.5097 - lr: 0.0010
Epoch 9/100
12/12 [==============================] - 12s 1s/step - loss: 1.6942 - accuracy: 0.5237 - lr:

In [13]:
train_loss, train_accuracy = model.evaluate(train_generator, steps=len(train_generator))

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")

12/12 [==============================] - 13s 1s/step - loss: 0.3048 - accuracy: 0.9164
Training Loss: 0.3048
Training Accuracy: 0.9164


In [34]:
model.save('/kaggle/working/myFruitclassifier001.h5')  

In [15]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_fruit_class(image_path, model_path='/kaggle/working/myFruitclassifier001'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array /= 255.0 
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    class_labels = ['apple fruit', 'banana fruit', 'cherry fruit', 'chickoo fruit', 'grapes fruit', 'kiwi fruit', 'mango fruit', 'orange fruit', 'strawberry fruit']
    predicted_class_label = class_labels[predicted_class_index]

    return predicted_class_label



In [21]:
predict_fruit_class('/kaggle/input/fruits-dataset-images/images/orange fruit/Image_10.jpg')

1/1 [==============================] - 0s 88ms/step


'orange fruit'

In [19]:
predict_fruit_class('/kaggle/input/validation-banana/Banana-Single.jpg')

1/1 [==============================] - 0s 90ms/step


'grapes fruit'

In [22]:
predict_fruit_class('/kaggle/input/fruits-dataset-images/images/strawberry fruit/Image_13.jpg')

1/1 [==============================] - 0s 94ms/step


'strawberry fruit'

In [40]:
def add_block_1(model, first_block=False):
    if first_block:
        model.add(Conv2D(32, (3, 3), padding='same', input_shape=(img_width, img_height, 3)))
    else:
        model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))

def add_block_2(model):
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))

def add_block_3(model):
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

In [41]:
# Initialize the model
my_model = Sequential()

# Repeat Blocks to add depth
add_block_1(my_model, first_block=True)
add_block_2(my_model)
add_block_1(my_model)
add_block_2(my_model)
add_block_3(my_model)
add_block_3(my_model)
add_block_3(my_model)

# Flatten the output to feed into dense layers
my_model.add(Flatten())

# Fully connected layers
my_model.add(Dense(512, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Dense(256, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Dense(128, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Dense(9, activation='softmax')) 

my_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(name='precision'),
                  tf.keras.metrics.Recall(name='recall'),
                  tf.keras.metrics.AUC(name='auc'),
                  tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')])
my_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_41 (Conv2D)          (None, 150, 150, 32)      896       
                                                                 
 leaky_re_lu_34 (LeakyReLU)  (None, 150, 150, 32)      0         
                                                                 
 batch_normalization_34 (Bat  (None, 150, 150, 32)     128       
 chNormalization)                                                
                                                                 
 conv2d_42 (Conv2D)          (None, 150, 150, 32)      9248      
                                                                 
 leaky_re_lu_35 (LeakyReLU)  (None, 150, 150, 32)      0         
                                                                 
 batch_normalization_35 (Bat  (None, 150, 150, 32)     128       
 chNormalization)                                     

In [42]:
num_epochs = 100
history = my_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    verbose=1,
)

Epoch 1/100
12/12 [==============================] - 28s 1s/step - loss: 3.1839 - accuracy: 0.1170 - precision: 0.1356 - recall: 0.0446 - auc: 0.4952 - categorical_accuracy: 0.1170
Epoch 2/100
12/12 [==============================] - 14s 1s/step - loss: 2.5609 - accuracy: 0.1086 - precision: 0.2093 - recall: 0.0251 - auc: 0.5242 - categorical_accuracy: 0.1086
Epoch 3/100
12/12 [==============================] - 13s 1s/step - loss: 2.3347 - accuracy: 0.1532 - precision: 0.3125 - recall: 0.0279 - auc: 0.5785 - categorical_accuracy: 0.1532
Epoch 4/100
12/12 [==============================] - 13s 1s/step - loss: 2.3115 - accuracy: 0.2061 - precision: 0.2821 - recall: 0.0306 - auc: 0.5952 - categorical_accuracy: 0.2061
Epoch 5/100
12/12 [==============================] - 13s 1s/step - loss: 2.1419 - accuracy: 0.2006 - precision: 0.3636 - recall: 0.0557 - auc: 0.6558 - categorical_accuracy: 0.2006
Epoch 6/100
12/12 [==============================] - 13s 1s/step - loss: 2.2210 - accuracy: 0.2

In [44]:
metrics = my_model.evaluate(train_generator, steps=len(train_generator))
train_loss, train_accuracy, train_precision, train_recall, train_auc, train_categorical_accuracy = metrics

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Training Precision: {train_precision:.4f}")
print(f"Training Recall: {train_recall:.4f}")
print(f"Training AUC: {train_auc:.4f}")
print(f"Training Categorical Accuracy: {train_categorical_accuracy:.4f}")

12/12 [==============================] - 13s 1s/step - loss: 2.9595 - accuracy: 0.3370 - precision: 0.5353 - recall: 0.2535 - auc: 0.7455 - categorical_accuracy: 0.3370
Training Loss: 2.9595
Training Accuracy: 0.3370
Training Precision: 0.5353
Training Recall: 0.2535
Training AUC: 0.7455
Training Categorical Accuracy: 0.3370
